In [1]:
#pip install gym
#pip install stable-baselines3
#pip install shimmy>=2.0

In [2]:
import warnings
warnings.filterwarnings("ignore", message=".*does not have valid feature names.*") #ignore warning when we pass numpy array without feature names instead of pandas df

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import gym
from gym import spaces
from stable_baselines3 import PPO
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('data/data_FE.csv')
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42766 entries, 0 to 42765
Data columns (total 256 columns):
 #    Column                               Dtype  
---   ------                               -----  
 0    nr._sessions.0                       float64
 1    total_km.0                           float64
 2    km_z3-4.0                            float64
 3    km_z5-t1-t2.0                        float64
 4    km_sprinting.0                       float64
 5    strength_training.0                  float64
 6    hours_alternative.0                  float64
 7    perceived_exertion.0                 float64
 8    perceived_trainingsuccess.0          float64
 9    perceived_recovery.0                 float64
 10   nr._sessions.1                       float64
 11   total_km.1                           float64
 12   km_z3-4.1                            float64
 13   km_z5-t1-t2.1                        float64
 14   km_sprinting.1                       float64
 15   strength_training

In [5]:
rl_features1 = ['km_sprinting_avg','hours_alternative_avg','perceived_exertion_avg','perceived_recovery_avg','stress_ratio_avg','high_zone_pct']

In [6]:
rl_features2 = ['km_sprinting_zcore_sum','hours_alternative_zcore_sum','perceived_exertion_zcore_sum','perceived_recovery_zcore_sum','stress_ratio_zcore_sum','high_zone_pct_zcore_sum']

In [7]:
rl_features = rl_features1 + rl_features2

In [8]:
print(rl_features)

['km_sprinting_avg', 'hours_alternative_avg', 'perceived_exertion_avg', 'perceived_recovery_avg', 'stress_ratio_avg', 'high_zone_pct', 'km_sprinting_zcore_sum', 'hours_alternative_zcore_sum', 'perceived_exertion_zcore_sum', 'perceived_recovery_zcore_sum', 'stress_ratio_zcore_sum', 'high_zone_pct_zcore_sum']


In [9]:
injury_pred_model_features = ['perceived_exertion.0', 'perceived_recovery.0', 'nr._sessions.0', 'total_km.0', 'stress_ratio.0']

In [10]:
df[rl_features].head()

,km_sprinting_avg,hours_alternative_avg,perceived_exertion_avg,perceived_recovery_avg,stress_ratio_avg,high_zone_pct,km_sprinting_zcore_sum,hours_alternative_zcore_sum,perceived_exertion_zcore_sum,perceived_recovery_zcore_sum,stress_ratio_zcore_sum,high_zone_pct_zcore_sum
0,0.24,0.416000,0.100,0.170000,0.591503,2.702703,2.267787,2.924673,1.632993,1.632993,2.841575,2.267787
1,0.24,0.416000,0.098,0.168000,0.586928,2.314815,2.267787,2.924673,1.643168,1.643168,2.841587,2.267787
2,0.24,0.416000,0.098,0.168000,0.586928,2.314815,2.267787,2.924673,1.643168,1.643168,2.841587,2.267787
3,0.24,0.416000,0.098,0.168000,0.586928,2.314815,2.267787,2.924673,1.643168,1.643168,2.841587,2.267787
4,0.20,0.346667,0.100,0.168333,0.596950,1.275510,2.267787,2.924673,1.825742,1.864675,2.436399,2.267787


In [11]:
df[rl_features].describe()

,km_sprinting_avg,hours_alternative_avg,perceived_exertion_avg,perceived_recovery_avg,stress_ratio_avg,high_zone_pct,km_sprinting_zcore_sum,hours_alternative_zcore_sum,perceived_exertion_zcore_sum,perceived_recovery_zcore_sum,stress_ratio_zcore_sum,high_zone_pct_zcore_sum
count,42766.000000,42766.000000,42766.000000,42766.000000,42766.000000,42766.000000,42766.000000,42766.000000,42766.000000,42766.000000,42766.000000,42766.000000
mean,0.094537,0.214775,0.319963,0.255480,1.298239,7.079865,0.737584,0.960041,1.934804,1.872339,2.467315,1.379550
std,0.386364,0.449783,0.202007,0.153484,0.994440,9.380236,1.158791,1.279243,0.824475,0.852498,0.762673,1.321029
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.135714,0.147143,0.853254,0.000000,0.000000,0.000000,1.578457,1.507557,2.339434,0.000000
50%,0.000000,0.000000,0.320000,0.225000,1.111222,3.246753,0.000000,0.000000,2.118802,2.049902,2.676220,2.267787
75%,0.083333,0.285714,0.485000,0.358333,1.514682,11.960133,2.267787,2.267787,2.508421,2.449490,2.873512,2.726969
max,13.024997,7.499999,0.979999,0.929999,28.433329,99.999989,3.207135,3.207135,3.207135,6.480741,3.207135,3.207135


In [12]:
df[df['injury']==0][rl_features].describe()

,km_sprinting_avg,hours_alternative_avg,perceived_exertion_avg,perceived_recovery_avg,stress_ratio_avg,high_zone_pct,km_sprinting_zcore_sum,hours_alternative_zcore_sum,perceived_exertion_zcore_sum,perceived_recovery_zcore_sum,stress_ratio_zcore_sum,high_zone_pct_zcore_sum
count,42183.000000,42183.000000,42183.000000,42183.000000,42183.000000,42183.000000,42183.000000,42183.000000,42183.000000,42183.000000,42183.000000,42183.000000
mean,0.093930,0.215072,0.318820,0.254873,1.294655,7.042192,0.734423,0.958979,1.931685,1.868837,2.465330,1.375294
std,0.385818,0.450775,0.201868,0.153488,0.993212,9.357339,1.157476,1.278962,0.826579,0.854741,0.765780,1.321164
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.135000,0.146667,0.851766,0.000000,0.000000,0.000000,1.576533,1.500000,2.337460,0.000000
50%,0.000000,0.000000,0.318000,0.224000,1.109524,3.181336,0.000000,0.000000,2.116743,2.045028,2.675957,2.267787
75%,0.083333,0.285714,0.483333,0.357500,1.510643,11.904762,2.267787,2.267787,2.507849,2.449490,2.873699,2.723524
max,13.024997,7.499999,0.979999,0.929999,28.433329,99.999989,3.207135,3.207135,3.207135,6.480741,3.207135,3.207135


In [13]:
df[df['injury']==1][rl_features].describe()

,km_sprinting_avg,hours_alternative_avg,perceived_exertion_avg,perceived_recovery_avg,stress_ratio_avg,high_zone_pct,km_sprinting_zcore_sum,hours_alternative_zcore_sum,perceived_exertion_zcore_sum,perceived_recovery_zcore_sum,stress_ratio_zcore_sum,high_zone_pct_zcore_sum
count,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000
mean,0.138428,0.193271,0.402644,0.299441,1.557532,9.805724,0.966357,1.036886,2.160493,2.125713,2.610953,1.687503
std,0.422049,0.370723,0.194953,0.146782,1.048462,10.568301,1.229964,1.298201,0.614329,0.620663,0.466390,1.275085
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.240179,0.180000,0.973196,0.000000,0.000000,0.000000,1.860669,1.840343,2.427599,0.000000
50%,0.000000,0.000000,0.427500,0.282000,1.252529,8.025682,0.000000,0.000000,2.252589,2.214037,2.694972,2.267787
75%,0.166667,0.300000,0.560000,0.399286,1.884157,15.929277,2.267787,2.267787,2.574865,2.541037,2.852088,2.842173
max,8.139998,3.917142,0.850000,0.863333,12.708694,68.421049,3.207135,3.207135,3.184156,3.207135,3.205061,3.203116


In [14]:
rl_features

['km_sprinting_avg',
 'hours_alternative_avg',
 'perceived_exertion_avg',
 'perceived_recovery_avg',
 'stress_ratio_avg',
 'high_zone_pct',
 'km_sprinting_zcore_sum',
 'hours_alternative_zcore_sum',
 'perceived_exertion_zcore_sum',
 'perceived_recovery_zcore_sum',
 'stress_ratio_zcore_sum',
 'high_zone_pct_zcore_sum']

In [15]:
weekly_mean_stats=df[rl_features+['injury']].mean().values
weekly_mean_stats

array([0.09453651, 0.21477512, 0.31996252, 0.25548027, 1.29823851,
       7.07986504, 0.73758439, 0.96004084, 1.93480382, 1.87233869,
       2.4673151 , 1.37955016, 0.01363232])

In [16]:
ml_model = joblib.load("models/bagging_model_balanced_oversampled.joblib")
scaler = joblib.load('models/scaler_oversampled.joblib')

In [17]:
unique_players = df['athlete_id'].unique()
train_players, test_players = train_test_split(
    unique_players, test_size=0.1, random_state=42
)

In [18]:
test_players

array([ 4, 63, 18,  0, 28, 73, 10, 34], dtype=int64)

In [19]:
df_train = df[df['athlete_id'].isin(train_players)].reset_index(drop=True)
df_test = df[df['athlete_id'].isin(test_players)].reset_index(drop=True)

In [20]:
df = df_train

In [21]:
len(df), len(df_test)

(39310, 3456)

# RL Agent

In [22]:
def reward_function(action, state):
    total_km, km_z34, km_z5, km_sprint, strength, alt_hrs = action

    ################################### Rewards
    reward = 0

    # Productive training reward
    prod_reward = 0.3 * km_z34 + 0.5 * km_z5 + 0.1 * total_km

    # Recovery practices reward
    rec_reward = 0.2 * alt_hrs + 0.4 * state['perceived_recovery_avg']

    reward += prod_reward + rec_reward

    ################################### Penalties

    # Deviation from baseline (Z-score) penalty with clipping
    z_penalty = 0
    for metric in ['km_sprinting', 'hours_alternative', 'perceived_exertion', 
                   'perceived_recovery', 'stress_ratio', 'high_zone_pct']:
        z = state.get(f'{metric}_zcore_sum', 0)
        if abs(z) > 3.0:
            penalty = -min(1.5 * abs(z), 5.0)  # Clip to max 5
            z_penalty += penalty

    reward += z_penalty

    # Stress penalty (scaled and capped)
    stress_penalty = -min((state['stress_ratio_avg'] - 1.3) * 1.0, 2.0) if state['stress_ratio_avg'] > 1.3 else 0
    reward += stress_penalty

    # Excessive high-intensity penalty (clip overuse)
    excess_hz_penalty = 0
    if state['high_zone_pct'] > 20:
        excess_hz_penalty = -min(0.05 * (state['high_zone_pct'] - 20) ** 2, 2.0)
        reward += excess_hz_penalty

    # Low recovery with high exertion penalty (quadratic growth)
    low_rec_high_ex_penalty = 0
    if (state['perceived_recovery_avg'] < 0.25) and (state['perceived_exertion_avg'] > 0.3):
        low_rec_high_ex_penalty = -((0.25 - state['perceived_recovery_avg']) * 
                                    (state['perceived_exertion_avg'] - 0.3)) * 4.0
        low_rec_high_ex_penalty = max(low_rec_high_ex_penalty, -2.0)
        reward += low_rec_high_ex_penalty

    # Injury risk proxy penalty (clip and scale down)
    injury_penalty = -min(state['perceived_recovery_avg'] * 2.5, 2.5)
    reward += injury_penalty

    return {
        'productive training reward': prod_reward,
        'good recovery practices reward': rec_reward,
        'deviation from normal baseline penalty': z_penalty,
        'high stress penalty': stress_penalty,
        'excessive high intensity penalty': excess_hz_penalty,
        'low recovery with high exertion penalty': low_rec_high_ex_penalty,
        'injury risk penalty': injury_penalty,
        'Total reward': reward
    }


In [23]:
class PlayerTrainingEnv(gym.Env):
    def __init__(self, player_data, ml_model, scaler, rl_features, weekly_mean_stats):
        super(PlayerTrainingEnv, self).__init__()
        
        self.player_data = player_data
        self.ml_model = ml_model
        self.rl_features = rl_features
        self.weekly_mean_stats = weekly_mean_stats
        self.scaler = scaler
        
        self.current_step = 0
        
        # state: weekly stats + injury prob
        #['km_sprinting_avg', 'hours_alternative_avg', 'perceived_exertion_avg', 'perceived_recovery_avg', 
        #'stress_ratio_avg', 'high_zone_pct', 'km_sprinting_zcore_sum', 'hours_alternative_zcore_sum', 
        #'perceived_exertion_zcore_sum', 'perceived_recovery_zcore_sum', 'stress_ratio_zcore_sum', 'high_zone_pct_zcore_sum']
        self.observation_space = spaces.Box(
            low=np.zeros(len(rl_features) + 1),
            high=np.array([15, 10, 1, 1, 5, 100,
                           5, 5, 5, 5, 5, 5,
                           1 
                          ]),
            dtype=np.float32
        )
        
        # action: trainig targets ['total_km', 'km_z3-4', 'km_z5-t1-t2', 'km_sprinting', 'strength_training', 'hours_alternative']
        self.action_space = spaces.Box(
            low=np.array([0, 0, 0, 0, 0, 0]),
            high=np.array([50, 30, 15, 5, 1, 5]),
            dtype=np.float32
        )
        
        

    def step(self, action):
        
        #injury risk pred based on latest day
        daily_features = self.player_data.iloc[self.current_step][[
            'perceived_exertion.0', 'perceived_recovery.0', 
            'nr._sessions.0', 'total_km.0', 'stress_ratio.0'
        ]].values.reshape(1, -1)
        
        daily_features = self.scaler.transform(daily_features)
        injury_prob = self.ml_model.predict_proba(daily_features)[0][1]
        

        
        #update state (use weekly averages from the current row)
        #['km_sprinting_avg', 'hours_alternative_avg', 'perceived_exertion_avg', 'perceived_recovery_avg', 
        #'stress_ratio_avg', 'high_zone_pct', 'km_sprinting_zcore_sum', 'hours_alternative_zcore_sum', 
        #'perceived_exertion_zcore_sum', 'perceived_recovery_zcore_sum', 'stress_ratio_zcore_sum', 'high_zone_pct_zcore_sum']
        features = self.rl_features + ["injury_prob"]
        row = self.player_data.iloc[self.current_step]
        values = [row[feat] for feat in self.rl_features] + [injury_prob]
        state_dict = dict(zip(features, values))
        
        reward_breakdown = reward_function(action, state_dict)
        reward = reward_breakdown['Total reward']
        
        next_state = np.array(values, dtype=np.float32)
        
        self.current_step += 1
        done = self.current_step >= len(self.player_data)
        
        return next_state, reward, done, {
            'weekly_targets': action,
            'injury_prob': injury_prob,
            'reward_breakdown': reward_breakdown 
        }

    def reset(self):
        self.current_step = 0
        return np.array(self.weekly_mean_stats)

In [24]:
#(self, player_data, ml_model, rl_features, weekly_mean_stats)
env = PlayerTrainingEnv(
    player_data=df, 
    ml_model=ml_model,
    scaler = scaler,
    rl_features=rl_features,
    weekly_mean_stats=weekly_mean_stats
)


model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=50_000) 
model.save("models/rl_runner_plan_advisor")

C:\Users\User\anaconda3\Lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
C:\Users\User\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 64   |
|    iterations      | 1    |
|    time_elapsed    | 31   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 60          |
|    iterations           | 2           |
|    time_elapsed         | 67          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.006112679 |
|    clip_fraction        | 0.0448      |
|    clip_range           | 0.2         |
|    entropy_loss         | -8.52       |
|    explained_variance   | 0.00238     |
|    learning_rate        | 0.0003      |
|    loss                 | 693         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00495    |
|    std                

-----------------------------------------
| time/                   |             |
|    fps                  | 54          |
|    iterations           | 12          |
|    time_elapsed         | 451         |
|    total_timesteps      | 24576       |
| train/                  |             |
|    approx_kl            | 0.004928087 |
|    clip_fraction        | 0.0215      |
|    clip_range           | 0.2         |
|    entropy_loss         | -8.71       |
|    explained_variance   | -0.00108    |
|    learning_rate        | 0.0003      |
|    loss                 | 241         |
|    n_updates            | 110         |
|    policy_gradient_loss | -0.00455    |
|    std                  | 1.03        |
|    value_loss           | 781         |
-----------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 54           |
|    iterations           | 13           |
|    time_elapsed         | 48

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 3.93e+04    |
|    ep_rew_mean          | -7.13e+04   |
| time/                   |             |
|    fps                  | 55          |
|    iterations           | 22          |
|    time_elapsed         | 806         |
|    total_timesteps      | 45056       |
| train/                  |             |
|    approx_kl            | 0.008330887 |
|    clip_fraction        | 0.0716      |
|    clip_range           | 0.2         |
|    entropy_loss         | -8.79       |
|    explained_variance   | 0.147       |
|    learning_rate        | 0.0003      |
|    loss                 | 216         |
|    n_updates            | 210         |
|    policy_gradient_loss | -0.00959    |
|    std                  | 1.05        |
|    value_loss           | 460         |
-----------------------------------------
------------------------------------------
| rollout/                |      

In [25]:
#model = PPO.load("models/rl_runner_plan_advisor")

In [26]:
def get_plan(rl_model, scaler, env, row):
    # Compute injury probability
    daily_features = row[[
        'perceived_exertion.0', 'perceived_recovery.0', 
        'nr._sessions.0', 'total_km.0', 'stress_ratio.0'
    ]].values.reshape(1, -1)
    daily_features = scaler.transform(daily_features)
    injury_prob = env.ml_model.predict_proba(daily_features)[0][1]

    # Construct the input state
    state = np.array([row[feat] for feat in env.rl_features] + [injury_prob], dtype=np.float32)

    # Predict action
    action, _states  = rl_model.predict(state)
    
    state_dict = dict(zip(env.rl_features + ["injury_prob"], state))
    
    reward_details = reward_function(action, state_dict)
    reward = reward_details['Total reward']

    return {
        "weekly_plan": {
            "total_km": action[0],
            "km_z3-4": action[1],
            "km_z5-t1-t2": action[2],
            "km_sprinting": action[3],
            "strength_training": int(round(action[4])),  # binary
            "hours_alternative": action[5]
        },
        "injury_prob": injury_prob,
        "total_reward": reward,
        "reward_breakdown": reward_details
    }


In [27]:
unseen_row = df_test.iloc[22]

# Get training plan
plan_info  = get_plan(model, scaler, env, unseen_row)

# Print results
print("Recommended training plan for today:")
print(plan_info["weekly_plan"])
print("Injury Prob:", plan_info["injury_prob"])
print("Reward Breakdown:", plan_info["reward_breakdown"])

Recommended training plan for today:
{'total_km': 0.0, 'km_z3-4': 0.25251466, 'km_z5-t1-t2': 2.2910116, 'km_sprinting': 0.5658301, 'strength_training': 0, 'hours_alternative': 0.0}
Injury Prob: 0.3878338814386089
Reward Breakdown: {'productive training reward': 1.2212601840496062, 'good recovery practices reward': 0.07199998497962952, 'deviation from normal baseline penalty': -4.734802007675171, 'high stress penalty': 0, 'excessive high intensity penalty': 0, 'low recovery with high exertion penalty': 0, 'injury risk penalty': -0.4499999061226845, 'Total reward': -3.8915417447686194}


In [28]:
unseen_row = df_test.iloc[5]

# Get training plan
plan_info  = get_plan(model, scaler, env, unseen_row)

# Print results
print("Recommended training plan for today:")
print(plan_info["weekly_plan"])
print("Injury Prob:", plan_info["injury_prob"])
print("Reward Breakdown:", plan_info["reward_breakdown"])

Recommended training plan for today:
{'total_km': 0.0, 'km_z3-4': 0.42804757, 'km_z5-t1-t2': 0.3037875, 'km_sprinting': 0.0, 'strength_training': 0, 'hours_alternative': 0.0}
Injury Prob: 0.0
Reward Breakdown: {'productive training reward': 0.28030802011489864, 'good recovery practices reward': 0.06666665673255921, 'deviation from normal baseline penalty': 0, 'high stress penalty': 0, 'excessive high intensity penalty': 0, 'low recovery with high exertion penalty': 0, 'injury risk penalty': -0.416666604578495, 'Total reward': -0.06969192773103716}
